# Comparison of the various model architectures tutorial

In the NeuralModel class, we propose several standard neural architectures for text classification. In particular, we include the well known `bert` architecture that usually allows for a significant improvement in NLP tasks.

* `RNN` are widely used for text processing.
* `CNN` or Convolutional Neural Networks are usually used for image classification tasks but give excellent results, comparable to the `RNN`, in our case.
* `Attentive` Neural Networks emerged recently for text processing and shows extremely promissing results. We therefore decided to include such models as part of Melusine and ease their used for email processing. We propose both an original attentive-based classifier as well as a wrap-up for a standard `Bert classifier`.

All our architectures follow the same general pattern: the email and header texts are embedded using one of the encoder listed above. The text vector is concatenated with a vector built using the email meta data (hour, email domain, email attachement ...)

In this tutorial we compare the different models' characteristics such as inference & training time, precision and architecture.

## Dataset preparation

The dataset preparation is developped in **tutorial 07: models**.

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";

# The GPU id to use, usually either "0" or "1", or "" to use CPU;
os.environ["CUDA_VISIBLE_DEVICES"]="";

In [9]:
# load inputs
import numpy as np
import pandas as pd
from copy import deepcopy
from sklearn.preprocessing import LabelEncoder

from melusine import load_email_data

# Text input 
df_emails_preprocessed = load_email_data(type="preprocessed")
df_emails_preprocessed['clean_text'] = df_emails_preprocessed['clean_header'] + " " + df_emails_preprocessed['clean_body']

# Metadata input
df_meta = pd.read_csv('/home/78169t/melusine/melusine/data/metadata.csv', encoding='utf-8', sep=';')

# Dataset
X = pd.concat([df_emails_preprocessed['clean_text'],df_meta],axis=1)
y = df_emails_preprocessed['label']
le = LabelEncoder()
y = le.fit_transform(y)

In [10]:
from melusine.nlp_tools.embedding import Embedding

pretrained_embedding = Embedding(
    tokens_column='tokens',
    workers=1,
    min_count=5
)

pretrained_embedding.train(df_emails_preprocessed) 

/home/78169t/.local/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## Models

In [11]:
import tensorflow.keras.backend as K
import time
from melusine.models.train import NeuralModel
from melusine.models.neural_architectures import cnn_model, rnn_model, transformers_model, bert_model

from sklearn.metrics import accuracy_score, classification_report

2022-03-29 11:41:25.234125: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-29 11:41:25.234177: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [13]:
#!pip install psutil
import psutil

In [14]:
def get_available_memory():
    return psutil.virtual_memory()._asdict()['available']

### RNN

`RNN` are traditionally used with textual data as they are specifically designed to handle sequentially structured data. Inputs are sequentially computed given a cell operation, generally a `LSTM` or `GRU` cell. At each step, the current input as well as the output from the previous step are used to compute the next hidden state. The proposed architecture includes a 2-layers bidirectional `GRU` network. The network last hidden state is used as the final sentence embedding.

<img src="./images/rnn-model.png" style="width:500px">

In [15]:
memory_start = get_available_memory()
RNN_model = NeuralModel(architecture_function=rnn_model,
                        pretrained_embedding=pretrained_embedding,
                        text_input_column="clean_text",
                        meta_input_list=['extension', 'dayofweek', 'hour', 'min','attachment_type'],
                        n_epochs=1)
training_start = time.time()
RNN_model.fit(X, y)
training_end = time.time()
RNN_memory = memory_start - get_available_memory()
RNN_memory = round(RNN_memory / 1e9 * 1024 , 1)
print('RNN is using {} Mb memory (RAM).'.format(str(RNN_memory)))

2022-03-29 11:43:33.476843: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-29 11:43:33.476910: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: slhdg002
2022-03-29 11:43:33.477977: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: slhdg002
2022-03-29 11:43:33.478081: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 418.87.1
2022-03-29 11:43:33.478147: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 418.87.1
2022-03-29 11:43:33.478158: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 418.87.1
2022-03-29 11:43:33.478446: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in 

1/1 [==============================] - 8s 8s/step - loss: 2.3979 - accuracy: 0.0250
RNN is using 308.0 Mb memory (RAM).


In [16]:
RNN_N_trainable_parameters = int(np.sum([K.count_params(p) for p in RNN_model.model.trainable_weights]))
RNN_N_non_trainable_parameters = int(np.sum([K.count_params(p) for p in RNN_model.model.non_trainable_weights]))
RNN_training_time = round(training_end - training_start, 2)
RNN_N_trainable_parameters, RNN_N_non_trainable_parameters, RNN_training_time

(346851, 0, 8.93)

In [17]:
y_res = []
inference_start = time.time()
for i in range(X.shape[0]):
    X_copy = deepcopy(X.loc[i:i, :])
    y_res.append(RNN_model.predict(X_copy))
y_res = le.inverse_transform(np.ravel(y_res))
inference_end = time.time()
RNN_inference_time = round((inference_end - inference_start)/len(y_res) * 1000, 2)
RNN_accuracy = accuracy_score(y_res, le.inverse_transform(y))
RNN_cls_report = classification_report(le.inverse_transform(y), y_res, output_dict=True, zero_division=0)

### CNN

`CNN` uses multiple filters to discriminate patterns in data. Such filters are assembled across the hidden layers to build more complex patterns and structures. The last layer should therefore capture a global and generic representation of the data. In our architecture, we use a two hidden layers `CNN` with respectively 200 filters for each hidden layer. The last hidden states are aggregated using a max pooling operation.

<img src="./images/cnn-model.png" style="width:500px">

In [18]:
memory_start = get_available_memory()
CNN_model = NeuralModel(architecture_function=cnn_model,
                        pretrained_embedding=pretrained_embedding,
                        text_input_column="clean_text",
                        meta_input_list=['extension', 'dayofweek', 'hour', 'min', 'attachment_type'],
                        n_epochs=1)
training_start = time.time()
CNN_model.fit(X, y)
training_end = time.time()
CNN_memory = memory_start - get_available_memory()
CNN_memory = round(CNN_memory / 1e9 * 1024 , 1)
print('CNN is using {} Mb memory (RAM).'.format(str(CNN_memory)))

1/1 [==============================] - 1s 1s/step - loss: 2.3947 - accuracy: 0.1000
CNN is using 66.9 Mb memory (RAM).


In [19]:
CNN_N_trainable_parameters = int(np.sum([K.count_params(p) for p in CNN_model.model.trainable_weights]))
CNN_N_non_trainable_parameters = int(np.sum([K.count_params(p) for p in CNN_model.model.non_trainable_weights]))
CNN_training_time =  round(training_end - training_start, 2)
CNN_N_trainable_parameters, CNN_N_non_trainable_parameters, CNN_training_time

(378961, 400, 1.43)

In [20]:
y_res = []
inference_start = time.time()
for i in range(X.shape[0]):
    X_copy = deepcopy(X.loc[i:i, :])
    y_res.append(CNN_model.predict(X_copy))
y_res = le.inverse_transform(np.ravel(y_res))
inference_end = time.time()
CNN_inference_time = round((inference_end - inference_start)/len(y_res) * 1000, 2)
CNN_accuracy = accuracy_score(y_res, le.inverse_transform(y))
CNN_cls_report = classification_report(le.inverse_transform(y), y_res, output_dict=True, zero_division=0)

### Transformer

#### Multi-heads Attention Classifier

Attentive-based neural networks are fairly new in the NLP community but results are extremely promising. They rely on the self-attention operation which computes hidden states as a weighted sum from the inputs. As the multiple filters in the `CNN` architecture, the multi-branch attention aggregate multiple attention operation to capture various properties from the input. Such operation is easily perform on GPU infrastructure. We propose an architecture inspired from previously introduced RNN and CNN architecture with a two layers multi-branch attention module follow by a max pooling operation.

<img src="./images/transformer-model.png" style="width:500px">

In [21]:
memory_start = get_available_memory()
Transformer_model = NeuralModel(architecture_function=transformers_model,
                                pretrained_embedding=pretrained_embedding,
                                text_input_column="clean_text",
                                meta_input_list=['extension', 'dayofweek', 'hour', 'min', 'attachment_type'],
                                n_epochs=1)
training_start = time.time()
Transformer_model.fit(X, y)
training_end = time.time()
Transformer_memory = memory_start - get_available_memory()
Transformer_memory = round(Transformer_memory / 1e9 * 1024 , 1)
print('Transformer is using {} Mb memory (RAM).'.format(str(Transformer_memory)))

1/1 [==============================] - 4s 4s/step - loss: 3.3700 - accuracy: 0.0750
Transformer is using 227.0 Mb memory (RAM).


In [22]:
Transformer_N_trainable_parameters = int(np.sum([K.count_params(p) for p in Transformer_model.model.trainable_weights]))
Transformer_N_non_trainable_parameters = int(np.sum([K.count_params(p) for p in Transformer_model.model.non_trainable_weights]))
Transformer_training_time =  round(training_end - training_start, 2)
Transformer_N_trainable_parameters, Transformer_N_non_trainable_parameters, Transformer_training_time

(1182561, 6900, 4.58)

In [23]:
y_res = []
inference_start = time.time()
for i in range(X.shape[0]):
    X_copy = deepcopy(X.loc[i:i, :])
    y_res.append(Transformer_model.predict(X_copy))
y_res = le.inverse_transform(np.ravel(y_res))
inference_end = time.time()
Transformer_inference_time = round((inference_end - inference_start)/len(y_res) * 1000, 2)
Transformer_accuracy = accuracy_score(y_res, le.inverse_transform(y))
Transformer_cls_report = classification_report(le.inverse_transform(y), y_res, output_dict=True, zero_division=0)

#### Bert Model

We also propose a wrap-up for the popular pre-trained `bert` architecture. `bert` architecture encodes every sentence tokens with a contextualized embeddings: each words embeddings depends from all words in the sentence. However, we only use the first sentence embedding, usually called the **classification token** in our classifier model.

<img src="./images/bert-model.png" style="width:500px">


Bert tokenizers and models can be downloaded here: https://huggingface.co/transformers/pretrained_models.html

Only Camembert and Flaubert are available now in Melusine.

### Camembert

In [25]:
memory_start = get_available_memory()
CamemBert_model = NeuralModel(architecture_function=bert_model,
                            pretrained_embedding=None,
                                text_input_column="clean_text",
                                meta_input_list=['extension', 'dayofweek', 'hour', 'min', 'attachment_type'],
                                n_epochs=1,
                        bert_tokenizer='jplu/tf-camembert-base',
                        bert_model='jplu/tf-camembert-base')
training_start = time.time()
CamemBert_model.fit(X, y)
training_end = time.time()
CamemBert_memory = memory_start - get_available_memory()
CamemBert_memory = round(CamemBert_memory / 1e9 * 1024 , 1)
print('CamemBert is using {} Mb memory (RAM).'.format(str(CamemBert_memory)))

Some layers from the model checkpoint at /home/public/NLP/bert_maif/huggingface/Camembert/tensorflow were not used when initializing TFCamembertModel: ['lm_head']
- This IS expected if you are initializing TFCamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFCamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFCamembertModel were initialized from the model checkpoint at /home/public/NLP/bert_maif/huggingface/Camembert/tensorflow.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertModel for predictions without further training.


1/1 [==============================] - 22s 22s/step - loss: 2.3404 - accuracy: 0.3250
CamemBert is using 3849.8 Mb memory (RAM).


In [26]:
CamemBert_N_trainable_parameters = int(np.sum([K.count_params(p) for p in CamemBert_model.model.trainable_weights]))
CamemBert_N_non_trainable_parameters = int(np.sum([K.count_params(p) for p in CamemBert_model.model.non_trainable_weights]))
CamemBert_training_time =  round(training_end - training_start, 2)
CamemBert_N_trainable_parameters, CamemBert_N_non_trainable_parameters, CamemBert_training_time

(110796963, 0, 28.99)

In [27]:
y_res = []
inference_start = time.time()
for i in range(X.shape[0]):
    X_copy = deepcopy(X.loc[i:i, :])
    y_res.append(CamemBert_model.predict(X_copy))
y_res = le.inverse_transform(np.ravel(y_res))
inference_end = time.time()
CamemBert_inference_time = round((inference_end - inference_start)/len(y_res) * 1000, 2)
CamemBert_accuracy = accuracy_score(y_res, le.inverse_transform(y))
CamemBert_cls_report = classification_report(le.inverse_transform(y), y_res, output_dict=True, zero_division=0)

### Flaubert

In [28]:
memory_start = get_available_memory()
FlauBert_model = NeuralModel(architecture_function=bert_model,
                            pretrained_embedding=None,
                                text_input_column="clean_text",
                                meta_input_list=['extension', 'dayofweek', 'hour', 'min', 'attachment_type'],
                                n_epochs=1,
                        bert_tokenizer='jplu/tf-flaubert-base-cased',
                        bert_model='jplu/tf-flaubert-base-cased')
training_start = time.time()
FlauBert_model.fit(X, y)
training_end = time.time()
FlauBert_memory = memory_start - get_available_memory()
FlauBert_memory = round(FlauBert_memory / 1e9 * 1024 , 1)
print('FlauBert is using {} Mb memory (RAM).'.format(str(FlauBert_memory)))

2022-03-29 11:48:25.473158: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Some layers from the model checkpoint at /home/public/NLP/bert_maif/huggingface/Flaubert/tensorflow were not used when initializing TFFlaubertModel: ['pred_layer_._proj']
- This IS expected if you are initializing TFFlaubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFFlaubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFFlaubertModel were initialized from the model checkpoint at /home/public/NLP/bert_maif/huggingface/Flaubert/tensorflow.
If your task is similar to the task the model

1/1 [==============================] - 40s 40s/step - loss: 3.0383 - accuracy: 0.1000
FlauBert is using 3894.8 Mb memory (RAM).


In [29]:
FlauBert_N_trainable_parameters = int(np.sum([K.count_params(p) for p in FlauBert_model.model.trainable_weights]))
FlauBert_N_non_trainable_parameters = int(np.sum([K.count_params(p) for p in FlauBert_model.model.non_trainable_weights]))
FlauBert_training_time =  round(training_end - training_start, 2)
FlauBert_N_trainable_parameters, FlauBert_N_non_trainable_parameters, FlauBert_training_time

(138408099, 0, 48.05)

In [30]:
y_res = []
inference_start = time.time()
for i in range(X.shape[0]):
    X_copy = deepcopy(X.loc[i:i, :])
    y_res.append(FlauBert_model.predict(X_copy))
y_res = le.inverse_transform(np.ravel(y_res))
inference_end = time.time()
FlauBert_inference_time = round((inference_end - inference_start)/len(y_res) * 1000, 2)
FlauBert_accuracy = accuracy_score(y_res, le.inverse_transform(y))
FlauBert_cls_report = classification_report(le.inverse_transform(y), y_res, output_dict=True, zero_division=0)

# Comparison of the model characteristics

In [31]:
parameters_list = ["Number of Parameters", "Memory Usage (Mb)", "Training time (s./epoch)",
                   "Inference time (ms./sample)", "Accuracy (%)"]
models_names = ['RNN', 'CNN', 'Transformers', 'CamemBert', 'FlauBert']

N_trainable_parameters = [RNN_N_trainable_parameters, CNN_N_trainable_parameters,
                          Transformer_N_trainable_parameters, CamemBert_N_trainable_parameters, FlauBert_N_trainable_parameters]

memory_usage = [RNN_memory, CNN_memory,
                Transformer_memory, CamemBert_memory, FlauBert_memory]

training_time = [RNN_training_time, CNN_training_time,
                 Transformer_training_time, CamemBert_training_time, FlauBert_training_time]

inference_time = [RNN_inference_time, CNN_inference_time,
                  Transformer_inference_time, CamemBert_inference_time, FlauBert_inference_time]

accuracy = [RNN_accuracy, CNN_accuracy,
            Transformer_accuracy, CamemBert_accuracy, FlauBert_accuracy]

data = [N_trainable_parameters, memory_usage, training_time, inference_time, accuracy]

In [32]:
def format_table(data, columns_header, row_header):
    row_format = "{:>15}|" * (len(row_header))
    row_format = "{:>30}|" + row_format
    space = ['']*len(row_header)
    space_format = "{:->15}+" * (len(row_header))
    space_format = "{:->30}+" + space_format
    print(row_format.format("", *row_header))
    for col, row in zip(columns_header, data):
        print(space_format.format("", *space))
        print(row_format.format(col, *row))

In [33]:
format_table(data, parameters_list, models_names)

                              |            RNN|            CNN|   Transformers|      CamemBert|       FlauBert|
------------------------------+---------------+---------------+---------------+---------------+---------------+
          Number of Parameters|         346851|         378961|        1182561|      110796963|      138408099|
------------------------------+---------------+---------------+---------------+---------------+---------------+
             Memory Usage (Mb)|          308.0|           66.9|          227.0|         3849.8|         3894.8|
------------------------------+---------------+---------------+---------------+---------------+---------------+
      Training time (s./epoch)|           8.93|           1.43|           4.58|          28.99|          48.05|
------------------------------+---------------+---------------+---------------+---------------+---------------+
   Inference time (ms./sample)|         107.22|          50.49|          76.64|          246.2|         

⚠️⚠️ **The metrics above are computed on a very small sample and may therefore be misleading. Please use your own dataset to compare extensively the models and their performances** ⚠️⚠️

# Classification Report

Please install plotly to plot the following graphs

``!pip install --upgrade plotly``

In [34]:
import plotly.graph_objs as go
import plotly as plotly
import plotly.express as px

In [35]:
def cls_report_2_df(cls_report, model_name):
    cls_report_df = pd.DataFrame.from_dict(cls_report)
    cls_report_df.drop(['accuracy', 'macro avg', 'weighted avg'], axis=1, inplace=True)
    cls_report_df = cls_report_df.T
    cls_report_df['class'] = cls_report_df.index
    cls_report_df['model'] = model_name
    return cls_report_df

In [36]:
models = ['RNN', 'CNN', 'Transformers', 'CamemBert', 'FlauBert']
cls_reports = [RNN_cls_report, CNN_cls_report, Transformer_cls_report, FlauBert_cls_report, CamemBert_cls_report]
ALL_cls_report_df = [cls_report_2_df(c, m) for (c, m) in zip(cls_reports, models)] 
ALL_cls_report_df = pd.concat(ALL_cls_report_df)

In [40]:
px.scatter(ALL_cls_report_df, 
             x="precision", 
             y="recall", 
             size='support', 
             color='model',
             hover_data=['class'],
             title="Precision and Recall given classes and models")

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([['adhesion'],
                                   ['autres'],
                                   ['bilan'],
                                   ['compte'],
                                   ['habitation'],
                                   ['modification'],
                                   ['regularisation'],
                                   ['resiliation'],
                                   ['sinistres'],
                                   ['succession'],
                                   ['vehicule']], dtype=object),
              'hovertemplate': ('model=RNN<br>precision=%{x}<br' ... '{customdata[0]}<extra></extra>'),
              'legendgroup': 'RNN',
              'marker': {'color': '#636efa',
                         'size': array([ 3.,  1.,  1.,  3.,  8.,  2.,  1.,  6.,  1.,  1., 13.]),
                         'sizemode': 'area',
                         'sizeref': 0.0325,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': 'RNN',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
                          0.325]),
              'xaxis': 'x',
              'y': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]),
              'yaxis': 'y'},
             {'customdata': array([['adhesion'],
                                   ['autres'],
                                   ['bilan'],
                                   ['compte'],
                                   ['habitation'],
                                   ['modification'],
                                   ['regularisation'],
                                   ['resiliation'],
                                   ['sinistres'],
                                   ['succession'],
                                   ['vehicule']], dtype=object),
              'hovertemplate': ('model=CNN<br>precision=%{x}<br' ... '{customdata[0]}<extra></extra>'),
              'legendgroup': 'CNN',
              'marker': {'color': '#EF553B',
                         'size': array([ 3.,  1.,  1.,  3.,  8.,  2.,  1.,  6.,  1.,  1., 13.]),
                         'sizemode': 'area',
                         'sizeref': 0.0325,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': 'CNN',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
                          0.325]),
              'xaxis': 'x',
              'y': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]),
              'yaxis': 'y'},
             {'customdata': array([['adhesion'],
                                   ['autres'],
                                   ['bilan'],
                                   ['compte'],
                                   ['habitation'],
                                   ['modification'],
                                   ['regularisation'],
                                   ['resiliation'],
                                   ['sinistres'],
                                   ['succession'],
                                   ['vehicule']], dtype=object),
              'hovertemplate': ('model=Transformers<br>precisio' ... '{customdata[0]}<extra></extra>'),
              'legendgroup': 'Transformers',
              'marker': {'color': '#00cc96',
                         'size': array([ 3.,  1.,  1.,  3.,  8.,  2.,  1.,  6.,  1.,  1., 13.]),
                         'sizemode': 'area',
                         'sizeref': 0.0325,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': 'Transformers',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              '

⚠️⚠️ **The metrics above are computed on a very small sample and may therefore be misleading. Please use your own dataset to compare extensively the models and their performances** ⚠️⚠️